# o'reillyのカサゴ深層学習の本

## MINST学習（誤差逆伝播）

## [目次](TableOfContents.ipynb)
- [計算グラフ](#計算グラフ)
- [ニューラルネットワーク](#ニューラルネットワーク)

## 参考
- https://github.com/oreilly-japan/deep-learning-from-scratch/blob/master/notebooks/
- [深層学習（deep learning） - 開発基盤部会 Wiki](https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E6%B7%B1%E5%B1%A4%E5%AD%A6%E7%BF%92%EF%BC%88deep%20learning%EF%BC%89)  
&gt; [ニューラルネットワーク](https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E3%83%8B%E3%83%A5%E3%83%BC%E3%83%A9%E3%83%AB%E3%83%8D%E3%83%83%E3%83%88%E3%83%AF%E3%83%BC%E3%82%AF)  
&gt; [ニューラルネットワーク（学習）](https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E3%83%8B%E3%83%A5%E3%83%BC%E3%83%A9%E3%83%AB%E3%83%8D%E3%83%83%E3%83%88%E3%83%AF%E3%83%BC%E3%82%AF%EF%BC%88%E5%AD%A6%E7%BF%92%EF%BC%89)  
&gt; [深層学習の誤差逆伝播法](https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E6%B7%B1%E5%B1%A4%E5%AD%A6%E7%BF%92%E3%81%AE%E8%AA%A4%E5%B7%AE%E9%80%86%E4%BC%9D%E6%92%AD%E6%B3%95)

## 深層学習の誤差逆伝播法
https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E6%B7%B1%E5%B1%A4%E5%AD%A6%E7%BF%92%E3%81%AE%E8%AA%A4%E5%B7%AE%E9%80%86%E4%BC%9D%E6%92%AD%E6%B3%95

### 計算グラフ
計算グラフで誤差逆伝播法を視覚的に理解するために使用する。

#### 計算グラフのレイヤの定義
基本的なレイヤの定義を行う。

##### 加算レイヤ

In [ ]:
# 加算レイヤ
class AddLayer:
    def __init__(self):
        pass

    def forward(self, x, y):
        # 加算
        out = x + y

        return out

    def backward(self, dout):
        # １を乗算
        dx = dout * 1
        dy = dout * 1

        return dx, dy

##### 乗算レイヤ

In [ ]:
# 乗算レイヤ
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None

    def forward(self, x, y):
        # 乗算
        self.x = x
        self.y = y                
        out = x * y

        return out

    def backward(self, dout):
        # 倍率を乗算
        dx = dout * self.y
        dy = dout * self.x

        return dx, dy

#### 計算グラフの逆伝播
以下の計算グラフの逆伝播で得られる値は、
- 連鎖率で微分を乗算していって得られた値で、
- 其々の値が１増えた時、その他の値に変更がない場合、最終結果に影響を与える大きさ。

##### 計算グラフの例１
以下の計算グラフの逆伝播で其々の値が１増えた時、  
その他の値に変更がない場合、最終結果に影響を与える大きさを計算。
<img src="../work/computational-graphs1.png" width="50%">

In [ ]:
apple = 100
apple_num = 2
tax = 1.1

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

# backward
dprice = 1 # 初期値は１（= dL/dL）
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print("- forward -")
print("price:", int(price))

print("- backward -")
print("dApple:", dapple)
print("dApple_num:", int(dapple_num))
print("dTax:", dtax)

##### 計算グラフの例２
以下の計算グラフの逆伝播で其々の値が１増えた時、  
その他の値に変更がない場合、最終結果に影響を与える大きさを計算。
<img src="../work/computational-graphs2.png" width="50%">

In [ ]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# layer
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)  # (1)
orange_price = mul_orange_layer.forward(orange, orange_num)  # (2)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)  # (3)
price = mul_tax_layer.forward(all_price, tax)  # (4)

# backward
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)  # (4)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)  # (3)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)  # (2)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)  # (1)

print("- forward -")
print("price:", int(price))

print("- backward -")
print("dApple:", dapple)
print("dApple_num:", int(dapple_num))
print("dOrange:", dorange)
print("dOrange_num:", int(dorange_num))
print("dTax:", dtax)

### ニューラルネットワーク

#### ニューラルネットワークのレイヤの定義
ココではニューラルネットワーク用のレイヤの定義を行う。
- ここでのポイントは、
  - ニューラルネットワークを計算グラフと捉え逆伝播を計算すると、
  - (数値) 微分をせずに勾配 → 重みを計算できる点。
- backwardの出力は、
  - forwardの入力の重み変数に対する勾配になる。
  - 特に、forwardでは、最後に誤差関数を用いるため、  
  誤差を０にするような重みを求めるための勾配になる。
- 従って、レイヤのクラスは以下のメソッドを持つ。
  - \_\_init\_\_(self, ...
  - def forward(self, ...
  - def backward(self, ...
- 各レイヤに実装されている処理は[コチラ](KasagoDL2.ipynb)を参考にすると良い。
- Import
  - [common/layers.py](Kasago/common/layers.py)

##### 中間層・出力層のレイヤ
- Affine  
中間層・出力層の重みとバイアスを処理するレイヤ

##### 中間層の活性化関数のレイヤ
- Sigmoid  
[中間層の活性化関数のシグモイド関数](KasagoDL1.ipynb)
- Relu  
[中間層の活性化関数のRelu関数](KasagoDL1.ipynb)

##### 出力層のソフトマックス関数のレイヤ
- SoftmaxWithLoss  
[出力層の活性化関数のソフトマックス関数](KasagoDL1.ipynb)と  
[交差エントロピー誤差の損失関数](KasagoDL2.ipynb)の両方が実装されている。

#### ニューラルネットワークの逆伝播
誤差逆伝播法で実装された２層ニューラルネットワーク。

##### 定義
- [common/layers.py](Kasago/common/layers.py)
- [common/gradient.py](Kasago/common/gradient.py)

In [ ]:
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定
import numpy as np
from kasago.common.layers import *
from kasago.common.gradient import numerical_gradient
from collections import OrderedDict

# ２層ニューラルネットワーク・クラス
class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # パラメタ（重みとバイアス）の初期化
        self.params = {}
        
        # ランダムな重み
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) 
        
        # バイアス（０
        self.params['b1'] = np.zeros(hidden_size)
        self.params['b2'] = np.zeros(output_size)

        # レイヤの生成
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        
        # 出力レイヤ
        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        # 推論
        
        # レイヤを順次取り出しながらforward実行
        for layer in self.layers.values():
            x = layer.forward(x)
        
        # 活性化関数（Relu）はlayers中に含まれる。
        
        return x
        
    # x:入力データ, t:教師データ
    def loss(self, x, t):
        # 勾配計算用関数
        
        # 推論（Affine.forward
        y = self.predict(x)
        
        # 損失関数（SoftmaxWithLoss.forward
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        # 正解率計算用関数
        
        # 推論（分類
        y = self.predict(x)
        
        # y, tを揃える
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        # 正解率の計算
        accuracy = np.sum(y == t) / float(x.shape[0])
        
        return accuracy

    # 学習（確認用）
    # x:入力データ, t:教師データ
    def numerical_gradient(self, x, t):
        # 数値微分による勾配計算により、
        # predictとLearnを行う。
        
        # 勾配計算用関数
        loss_W = lambda W: self.loss(x, t)
        
        # 勾配計算
        # ヤヤコシイが
        #  - ココのnumerical_gradientはself.numerical_gradientじゃない。
        #  - なお、self.predictは、self.lossの中で実行されている。
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
    
    # 学習（高速版）
    # x:入力データ, t:教師データ
    def gradient(self, x, t):
        # 誤差逆伝播法による勾配計算により、
        # predictとLearnを行う。
        
        # 勾配の計算
        
        # forward（predict
        self.loss(x, t)

        # backward（Learn
        dout = 1
        
        # 出力レイヤの逆順再生
        dout = self.lastLayer.backward(dout)
        
        # 中間レイヤの逆順再生
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # パラメタ（重みとバイアス）の更新
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads

##### 実行

###### 数値微分と誤差逆伝播法の差
- 数値微分と誤差逆伝播法で算出した勾配の差を確認
- かなり小さい値になっていればOK（e^-10 前後）
- Import
  - [dataset/mnist.py](Kasago/dataset/mnist.py)

In [ ]:
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定
import numpy as np
from kasago.dataset.mnist import load_mnist

# データの読み込み
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

# ２層ニューラルネット
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

# バッチサイズ３
x_batch = x_train[:3]
t_batch = t_train[:3]

# 数値微分と誤差逆伝播法で勾配算出
grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

# 算出した勾配の差を確認（コレを勾配確認と呼ぶ）
for key in grad_numerical.keys():
    diff = np.average(np.abs(grad_backprop[key] - grad_numerical[key]))
    print(key + ":" + str(diff))

###### 誤差逆伝播法でMISNTの画像認識
- Import
  - [dataset/mnist.py](Kasago/dataset/mnist.py)

In [ ]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from kasago.dataset.mnist import load_mnist

# データの読み込み
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

# ２層ニューラルネット
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

# ハイパーパラメタ

# 繰り返し回数を適宜設定する
iters_num = 10000
# 学習率
learning_rate = 0.1

# 訓練データサイズ
train_size = x_train.shape[0]
# バッチサイズ
batch_size = 100
# サブセット数（イテレーション数）
iter_per_epoch = max(train_size / batch_size, 1)

# 結果を格納する変数
train_loss_list = []
train_acc_list = []
test_acc_list = []

for i in range(iters_num): # 繰り返し回数
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 学習
    
    # 数値微分は、都度、推論が必要なので遅過ぎて使えない。
    # grad = network.numerical_gradient(x_batch, t_batch)
    # 誤差逆伝播法の実用可能な高速版
    grad = network.gradient(x_batch, t_batch)
    
    # パラメタ（重みとバイアス）の更新
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    # 損失の記録
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:# エポック毎
        
        # 正解率の計算
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        # 正解率の記録
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)
        
print("train and test have been completed.")